Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# Heterogeneous graph learning on IPUs

Many real-world graphs are heterogeneous, meaning single node and edge features are insufficient to capture all the information in the graph, leading to graphs which have a different node types and different edge types between those nodes. This comes with a few considerations, how do we construct a model suitable for training with heterogeneous graph data and how do we create mini-batches from this data. We will answer both of those questions, with a focus of looking at how we do these on Graphcore IPUs to enable accelerating heterogeneous graph learning workloads.

In this tutorial you will learn how to:

- Use a couple of PyTorch Geometric approaches to Heterogeneous graph learning and how to run them on the IPU.
- Understand how to sample heterogeneous graphs with a fixed size suitable for the IPU.

While this tutorial will cover enough of the basics of GNNs, PyTorch Geometric and PopTorch
for you to start developing and porting your GNN applications to the IPU;
the following resources can be used to complement your understanding of:

- PopTorch : [Introduction to PopTorch - running a simple model](https://github.com/graphcore/tutorials/tree/master/tutorials/pytorch/basics);
- GNNs : [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/)
- PyTorch Geometric (PyG): [Heterogeneous Graph Documentation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html)

## Running on Paperspace

The Paperspace environment lets you run this notebook with no set up. To improve your experience we preload datasets and pre-install packages, this can take a few minutes, if you experience errors immediately after starting a session please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).

Requirements:

* Python packages installed with `pip install -r ./requirements.txt`


In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext gc_logger` from any cell.

In [53]:
# Make imported python modules automatically reload when the files are changed
# needs to be before the first import.
%load_ext autoreload
%autoreload 2
# TODO: remove at the end of notebook development

In [2]:
%pip install -q -r ./requirements.txt
from examples_utils import notebook_logging
%load_ext gc_logger

Note: you may need to restart the kernel to use updated packages.


cp: failed to access '/root/.ipython/extensions': Permission denied


ModuleNotFoundError: No module named 'gc_logger'

And for compatibility with the Paperspace environment variables we will do the following:

In [3]:
import os

executable_cache_dir = (
    os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/pyg-packing"
)
dataset_directory = os.getenv("DATASETS_DIR", "data")

Now we are ready to start!

## Introduction to heterogeneous graphs

Heterogeneous graphs are graphs with different types of nodes and edges, appropriate when having a single node or edge feature for the whole graph doesn't capture all the information of the graph, for example because different nodes have a different amount of features. Let's first load a heterogeneous graph dataset from PyTorch Geometric and then go on to see how the construction of the model differs when dealing with heterogeneous graph data.

### Loading a heterogeneous graph dataset

In this tutorial we will use the [IMDB](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.IMDB.html) from PyTorch Geometric:

In [30]:
from torch_geometric.datasets import IMDB

dataset = IMDB(root=f"{dataset_directory}/IMDB")

This dataset is a single large heterogeneous graph,made up of three node types **movie**, **director** and **actor**, each with their own sets of features (`x`). These nodes are connected by two edge types, **movie to director** and **movie to actor**, with the reverse of those edges also present.

![IMDB_dataset.jpg](static/IMDB_dataset.jpg)

let's take a look at it in PyTorch Geometric:

In [31]:
data = dataset[0]
data

HeteroData(
  movie={
    x=[4278, 3066],
    y=[4278],
    train_mask=[4278],
    val_mask=[4278],
    test_mask=[4278]
  },
  director={ x=[2081, 3066] },
  actor={ x=[5257, 3066] },
  (movie, to, director)={ edge_index=[2, 4278] },
  (movie, to, actor)={ edge_index=[2, 12828] },
  (director, to, movie)={ edge_index=[2, 4278] },
  (actor, to, movie)={ edge_index=[2, 12828] }
)

The **movie** node type is the target for any training we will do. Taking a look at the labels you can see they are one of three classes, these correspond to the genre of the movie, action, comedy or drama.

In [32]:
import torch

classes = torch.unique(data["movie"].y)
num_classes = len(classes)
classes, num_classes

(tensor([0, 1, 2]), 3)

For the purposes of this tutorial, we will select only a fraction of this dataset. We will cover proper sampling approaches in the `Fixed size heterogeneous data loading` section.

In [35]:
from torch_geometric.transforms import RemoveIsolatedNodes

data = data.subgraph({"movie": torch.arange(0, 1000)})
data = RemoveIsolatedNodes()(data)
data

HeteroData(
  movie={
    x=[1000, 3066],
    y=[1000],
    train_mask=[1000],
    val_mask=[1000],
    test_mask=[1000]
  },
  director={ x=[482, 3066] },
  actor={ x=[1469, 3066] },
  (movie, to, director)={ edge_index=[2, 1000] },
  (movie, to, actor)={ edge_index=[2, 2998] },
  (director, to, movie)={ edge_index=[2, 1000] },
  (actor, to, movie)={ edge_index=[2, 2998] }
)

Now we have some understanding of what a heterogeneous graph looks like in PyTorch Geometric, let's next understand how we would construct a model to be able to learn from a heterogeneous graph.

## Creating Heterogeneous GNNs

PyTorch Geometric provides three ways to create a model for heterogeneous graph data, we will take a look at each in turn and understand any modifications to make to enable these models to run on the IPU. Each approach is taken from the [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html). Here we will cover the changes when enabling running these approaches on the IPU. For more detail on the approaches themselves we recommend reading the documentation directly.

### Converting a GNN model

The first approach we will look at is converting a PyTorch Geometric GNN model to a model for heterogeneous graphs using the `torch_geometric.nn.to_hetero()` transformation.

We will only cover the basics here to enable running on the IPU, but if you are interested in learning more about this approach see the [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html#automatically-converting-gnn-models).

To begin with let's create a PyTorch Geometric GNN model, comprising of a couple of convolution layers:

In [36]:
import torch
from torch_geometric.nn import SAGEConv


class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), 64)
        self.conv2 = SAGEConv((-1, -1), 64)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

Now we can use the `to_hetero()` transformation to transform this GNN model into a heterogeneous model:

In [37]:
from torch_geometric.nn import to_hetero

# Initialize the model
model = Model()
# Convert the model to a heterogeneous model
model = to_hetero(model, data.metadata(), aggr='sum')
model

GraphModule(
  (conv1): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 64, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 64, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
  )
  (conv2): ModuleDict(
    (movie__to__director): SAGEConv((-1, -1), 64, aggr=mean)
    (movie__to__actor): SAGEConv((-1, -1), 64, aggr=mean)
    (director__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
    (actor__to__movie): SAGEConv((-1, -1), 64, aggr=mean)
  )
)

You can now see that we have a convolution layer for each edge type, which has enabled this model to do message passing on a heterogeneous graph. The model will now expect a dictionary of node and edge types as inputs.

Notice how we set the convolution layer `in_channels` to `-1`. This allows PyTorch Geometric to use lazy initialization based on the input dimensions, which means we don't need to manually specify the dimensions for each node type. We can then perform this lazy initialization on the CPU as follows:

In [38]:
# Initialize lazy modules.
with torch.no_grad():
    out = model(data.x_dict, data.edge_index_dict)

To run your model using PyTorch Geometric on the IPU, the model will need to target PopTorch and will require a number of changes.

The first change is to move the loss function inside the `forward` method of the model. We can do this by creating a simple module that wraps the transformed heterogeneous model, that includes the loss calculation:

In [39]:
import torch.nn.functional as F


class ModelWithLoss(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x_dict, edge_index_dict, target=None, train_mask=None):
        out = self.model(x_dict, edge_index_dict)
        if self.training:
            target = torch.where(train_mask, target, -100)
            loss = F.cross_entropy(out['movie'], target)
            return out, loss
        return out

# Include loss in model
model = ModelWithLoss(model)

Now our model is ready for training with PopTorch on IPUs.

In the normal way we can wrap our model in `poptorch.trainingModel`:

In [40]:
import poptorch

# Set up training
model.train()

# Initialise model and convert the model to a poptorch model
opts = poptorch.Options().enableExecutableCaching(executable_cache_dir)
optim = poptorch.optim.Adam(model.parameters(), lr=0.01)
poptorch_model = poptorch.trainingModel(model, options=opts, optimizer=optim)

And run the training loop. Note the backward pass and optimizer step is handled by PopTorch automatically so does not need to be included.

In [45]:
# Train
for _ in range(3):
    out, loss = poptorch_model(data.x_dict,
                               data.edge_index_dict,
                               target=data['movie'].y,
                               train_mask=data['movie'].train_mask)
    print(f"{loss = }")

loss = tensor(0.0003)
loss = tensor(0.0002)
loss = tensor(9.7318e-05)


Here we have seen how to create a heterogeneous GNN using the `to_hetero()` transformation and start training on the IPU. An alternative approach is to use the `HeteroConv` layer which we will see next.

### Using the Heterogeneous Convolution Wrapper

Another approach to doing heterogeneous graphs with PyTorch Geometric is to use the `torch_geometric.nn.HeteroConv` layer. This gives more flexibility than the above approach allowing each edge type to use a different message passing layer. For more details on this approach, see the [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html#using-the-heterogeneous-convolution-wrapper).

Now let's take the steps to enable using this approach for the IPU. As normal we first move the loss function within the model, passing the mask and labels to the `forward` method. Below you can see a simple model using the `HeteroConv` layer with the loss function moved inside the `forward` method, ready for running on the IPU.

In [46]:
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv, Linear


class HeteroGNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                ('movie', 'to', 'director'): SAGEConv((-1, -1), hidden_channels),
                ('director', 'to', 'movie'): SAGEConv((-1, -1), hidden_channels),
                ('movie', 'to', 'actor'): GATConv((-1, -1), hidden_channels, add_self_loops=False),
                ('actor', 'to', 'movie'): GATConv((-1, -1), hidden_channels, add_self_loops=False),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self,
                x_dict,
                edge_index_dict,
                target=None,
                train_mask=None):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        out = self.lin(x_dict['movie'])

        if self.training:
            target = torch.where(train_mask, target, -100)
            loss = F.cross_entropy(out, target)
            return out, loss
        return out

model = HeteroGNN(hidden_channels=64,
                  out_channels=num_classes,
                  num_layers=2)

In the same way as before we set the convolution layer `in_channels` to `-1`. We can then perform the lazy initialization on the CPU again as follows:

In [47]:
# Initialize lazy modules.
with torch.no_grad():
    out = model(data.x_dict,
                data.edge_index_dict,
                target=data['movie'].y,
                train_mask=data['movie'].train_mask)

We wrap the model in `poptorch.trainingModel`:

In [48]:
# Set up training
model.train()

# Initialise model and convert the model to a poptorch model
opts = poptorch.Options().enableExecutableCaching(executable_cache_dir)
optim = poptorch.optim.Adam(model.parameters(), lr=0.01)
poptorch_model = poptorch.trainingModel(model, options=opts, optimizer=optim)

And perform the training loop:

In [50]:
# Train
for _ in range(3):
    out, loss = poptorch_model(data.x_dict,
                               data.edge_index_dict,
                               target=data['movie'].y,
                               train_mask=data['movie'].train_mask)
    print(f"{loss = }")

loss = tensor(0.5073)
loss = tensor(0.3215)
loss = tensor(0.1516)


We have now seen two approaches to creating heterogeneous GNNs ready for the IPU using PyTorch Geometric. We will next look at the final approach, using heterogeneous operators.

### Using Heterogeneous operators

The final approach PyTorch Geometric provides to create a heterogeneous GNN model is to use operators specifically designed for heterogeneous graphs. These layers can be used as normal, taking care to do the normal steps mentioned previously to enable running on IPUs: moving the loss inside the model, wrapping the model in `poptorch.trainingModel` and removing the call to the backward pass and optimizer step.

See the [PyTorch Geometric Documenation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html#deploy-existing-heterogeneous-operators) for more information on this approach.

## Fixed size heterogeneous data loading

As real-world heterogeneous graphs can be quite large, it may often be appropriate to move from full-batch training to mini-batch training using some form of sampling. PyTorch Geometric provides a range of samplers suitable for heterogeneous graphs, for example the `NeighborLoader` which we will look at below.

When moving from full-batch to mini-batch on the IPU, one must consider the sizes of the mini-batches. The IPU uses ahead-of-time compilation, which means all mini-batches must be the same size, outlined in previous tutorials (TODO). In the homogeneous graph case, making our mini-batches fixed size is relatively trivial, adding padding to make the nodes and edges up to a fixed size. This becomes more complex with heterogeneous graphs when there are different node and edge types.

Let's create an instance of the PyTorch Geometric `NeighborLoader` with our dataset, and see what the first mini-batch looks like:

In [51]:
# As normal

from torch_geometric.loader import NeighborLoader


train_loader = NeighborLoader(
    data,
    num_neighbors=[5] * 2,
    batch_size=128,
    input_nodes=('movie', data['movie'].train_mask),
)

next(iter(train_loader))

/nethome/adams/venvs/3.3.0+1361-EA.2/3.3.0+1361_poptorch/lib/python3.8/site-packages/torch_geometric/sampler/neighbor_sampler.py:58: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


HeteroData(
  movie={
    x=[486, 3066],
    y=[486],
    train_mask=[486],
    val_mask=[486],
    test_mask=[486],
    n_id=[486],
    input_id=[87],
    batch_size=87
  },
  director={
    x=[77, 3066],
    n_id=[77]
  },
  actor={
    x=[227, 3066],
    n_id=[227]
  },
  (movie, to, director)={
    edge_index=[2, 212],
    e_id=[212]
  },
  (movie, to, actor)={
    edge_index=[2, 596],
    e_id=[596]
  },
  (director, to, movie)={
    edge_index=[2, 87],
    e_id=[87]
  },
  (actor, to, movie)={
    edge_index=[2, 261],
    e_id=[261]
  }
)

To make up this mini-batch to a fixed size, we could simply pad the nodes and edges of each node and edge type to a particular value. We have seen in other tutorials how we can make a mini-batch fixed size for homogeneous graphs, for example in the case of neighbor loading see TODO. For heterogeneous graphs we have some additional considerations to make. TODO

In [54]:
from poptorch_geometric import FixedSizeOptions

fixed_size_options = FixedSizeOptions(
    num_nodes=1000,
    num_edges=1000,
)
print(fixed_size_options)

FixedSizeOptions(num_nodes=1000 (At least one node reserved for padding), num_edges=1000 (At least one edge reserved for padding), num_graphs=2 (At least one graph reserved for padding), node_pad_value=0.0, edge_pad_value=0.0, graph_pad_value=0.0, pad_graph_defaults={})


Here you can see the fixed sizes that will be appropriate for the neighbor loading. Now we can use these sizes to create a fixed size version of the `NeighborLoader` the `poptorch_geometric.FixedSizeNeighborLoader` that will do the same sampling but produce fixed size mini-batches.

TODO: See sampling tutorial.

In [ ]:
from poptorch_geometric import FixedSizeNeighborLoader, OverSizeStrategy


fixed_size_train_loader = FixedSizeNeighborLoader(
    data,
    num_neighbors=[15] * 2,
    batch_size=128,
    input_nodes=('movie', data['movie'].train_mask),
    fixed_size_options=fixed_size_options,
    over_size_behaviour=OverSizeStrategy.TRIM_NODES_AND_EDGES
)

next(iter(train_loader))

You can see that now the nodes and edges of each node and edge type are the same size and so suitable for using with the IPU.

Note how we have set `over_size_behaviour=OverSizeBehaviour.TRIM_NODES_AND_EDGES`. Unfortunately we don't know ahead of time whether we have allocated enough space for the padding, therefore we can enable trimming any excess nodes from our samples in the case the mini-batches are greater than our specified sizes.

There may be a lot of wasted space in this mini-batch as we have set the same number of nodes and edges to pad to for all node and edge types. We can be more specific and set a different number for each type.

In [103]:
fixed_size_options = FixedSizeOptions(
    num_nodes={
        "movies": 500,
        "directors": 100,
        "actors": 300
    },
    num_edges={
        ("movies", "to", "directors"): 100,
        ("movies", "to", "actors"): 200,
        ("directors", "to", "movies"): 100,
        ("actors", "to", "movies"): 200,
    }
)
print(fixed_size_options)

This can become quite complex so instead we can use the original non-fixed-size neighbour loader to give us an estimate of the fixed size options suitable for this loader. This will sample from the non-fixed-size loader, and produce fixed-size options which will have different numbers of nodes and edges to pad to for each node and edge type. This can help make your mini-batches contain less padded nodes and edges.

In [ ]:
# TODO: Enable this in fixed size options

fixed_size_options = FixedSizeOptions.from_loader(train_loader)
print(fixed_size_options)

And use this to create the `FixedSizeNeighborLoader`:

In [ ]:
fixed_size_train_loader = FixedSizeNeighborLoader(
    data,
    num_neighbors=[15] * 2,
    batch_size=128,
    input_nodes=('movie', data['movie'].train_mask),
    fixed_size_options=fixed_size_options,
    over_size_behaviour=OverSizeStrategy.TRIM_NODES_AND_EDGES
)

next(iter(train_loader))

Again, you can see the mini-batches produced are fixed size and so suitable for using with the IPU.

## Conclusion

In this tutorial we have learnt how to train heterogeneous GNNs on the IPU using PyTorch Geometric.

You should now have a good understanding of:
 - the approaches PyTorch Geometric provides to create heterogeneous GNN models
 - how to run the model produced by each approach on the IPU
 - how to achieve fixed size mini-batches of heterogeneous graphs suitable for the IPU.

Additional resources which may help you understand Heterogeneous Graph Learning can be found in the [PyTorch Geometric documentation](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/heterogeneous.html)